# EDX Dashboard Backend (EDA)⛏:
This is a Jupyter Notebook file that features the exploratory data analysis (EDA) steps taken to create the CSV files and SQL queries that fuel the Udemy Dashboard. The tools used include Python and SQL Lite.

In [2]:
import sqlite3
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
edx_FILEPATH = "../data/edx_courses.csv"
edx_courses = pd.read_csv(edx_FILEPATH)
edx_courses.tail(3)

,title,summary,n_enrolled,course_type,institution,instructors,Level,subject,language,subtitles,course_effort,course_length,price,course_description,course_syllabus,course_url
972,Computational Neuroscience: Neuronal Dynamics ...,This course explains the mathematical and comp...,"11,246",Self-paced on your time,École polytechnique fédérale de Lausanne,Wulfram Gerstner,Advanced,Biology & Life Sciences,English,English,4–6 hours per week,6 Weeks,FREE-Add a Verified Certificate for $139 USD,What happens in your brain when you make a dec...,Textbook: Neuronal Dynamics - from single neur...,https://www.edx.org/course/computational-neuro...
973,Cities and the Challenge of Sustainable Develo...,What is a sustainable city? Learn the basics h...,"8,775",Self-paced on your time,SDG Academy,Jeffrey D. Sachs,Introductory,Environmental Studies,English,English,1–2 hours per week,1 Weeks,FREE-Add a Verified Certificate for $25 USD,"According to the United Nations, urbanization ...",Module 1: Introduction to the SDGsProfessor Je...,https://www.edx.org/course/cities-and-the-chal...
974,MathTrackX: Special Functions,"Understand trigonometric, exponential and loga...",NaN,Self-paced on your time,University of Adelaide,Dr David Butler,Introductory,Math,English,English,3–6 hours per week,4 Weeks,FREE-Add a Verified Certificate for $79 USD,This course is part two of the MathTrackX XSer...,NaN,https://www.edx.org/course/mathtrackx-special-...


In [4]:
edx_courses.dtypes

title                 object
summary               object
n_enrolled            object
course_type           object
institution           object
instructors           object
Level                 object
subject               object
language              object
subtitles             object
course_effort         object
course_length         object
price                 object
course_description    object
course_syllabus       object
course_url            object
dtype: object

## Data Cleaning

In [36]:
# sum of null values in every column
edx_courses.isna().sum()

title                   0
summary                53
n_enrolled            120
course_type             0
institution             0
instructors             3
Level                   0
subject                 0
language                0
subtitles               3
course_effort           0
course_length           0
price                   0
course_description     40
course_syllabus       558
course_url              0
dtype: int64

In [37]:
# clean 'price' column
edx_courses['price'] = edx_courses['price'].str.replace('FREE-Add a Verified Certificate for ', '')
edx_courses['price'] = edx_courses['price'].str.replace('$', '')
edx_courses['price'] = edx_courses['price'].str.replace(' USD', '')
edx_courses['price'] = edx_courses['price'].astype(float)
edx_courses['price']

<ipython-input-37-8add241d36b6>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  edx_courses['price'] = edx_courses['price'].str.replace('$', '')


0       49.0
1       49.0
2       90.0
3      199.0
4      249.0
       ...  
970     25.0
971    139.0
972    139.0
973     25.0
974     79.0
Name: price, Length: 975, dtype: float64

In [38]:
# clean 'n_enrolled' column
edx_courses['n_enrolled'] = edx_courses['n_enrolled'].str.replace(',', '')
edx_courses['n_enrolled'] = edx_courses['n_enrolled'].fillna(0)
edx_courses['n_enrolled'] = edx_courses['n_enrolled'].astype(int)
edx_courses['n_enrolled'].head()

0     124980
1     293864
2    2442271
3     129555
4      81140
Name: n_enrolled, dtype: int32

In [39]:
# clean 'course_length' column
edx_courses['course_length'] = edx_courses['course_length'].str.replace('Weeks', '')
edx_courses['course_length'] = edx_courses['course_length'].astype(int)
edx_courses['course_length'].head()

0     2
1     7
2    12
3    13
4     4
Name: course_length, dtype: int32

In [40]:
# fill null values
edx_courses['summary'] = edx_courses['summary'].fillna('Unlisted')
edx_courses['instructors'] = edx_courses['instructors'].fillna('Unlisted')
edx_courses['subtitles'] = edx_courses['subtitles'].fillna('English')
edx_courses['course_description'] = edx_courses['course_description'].fillna('Unlisted')
edx_courses['course_syllabus'] = edx_courses['course_syllabus'].fillna('Unlisted')

In [41]:
# sum of null values in every column(after filling)
edx_courses.isna().sum()

title                 0
summary               0
n_enrolled            0
course_type           0
institution           0
instructors           0
Level                 0
subject               0
language              0
subtitles             0
course_effort         0
course_length         0
price                 0
course_description    0
course_syllabus       0
course_url            0
dtype: int64

In [42]:
edx_courses.dtypes

title                  object
summary                object
n_enrolled              int32
course_type            object
institution            object
instructors            object
Level                  object
subject                object
language               object
subtitles              object
course_effort          object
course_length           int32
price                 float64
course_description     object
course_syllabus        object
course_url             object
dtype: object

## Data transformation

In [43]:
# establish categories used in column: ['Advanced', 'Intermediate', 'Introductory']
edx_courses['Level'] = edx_courses['Level'].astype('category')
print(edx_courses['Level'].value_counts())
print()

# transform categories to numerical values
edx_courses['Level'] = edx_courses['Level'].cat.codes
print(edx_courses['Level'].value_counts())

Introductory    621
Intermediate    267
Advanced         87
Name: Level, dtype: int64

2    621
1    267
0     87
Name: Level, dtype: int64


In [44]:
# establish categories used in column: ['Instructor-led on a course schedule', 'Self-paced on your time']
edx_courses['course_type'] = edx_courses['course_type'].astype('category')
print(edx_courses['course_type'].value_counts())
print()

# transform categories to numerical values
edx_courses['course_type'] = edx_courses['course_type'].cat.codes
print(edx_courses['course_type'].value_counts())

Self-paced on your time                917
Instructor-led on a course schedule     58
Name: course_type, dtype: int64

1    917
0     58
Name: course_type, dtype: int64


In [46]:
edx_courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               975 non-null    object 
 1   summary             975 non-null    object 
 2   n_enrolled          975 non-null    int32  
 3   course_type         975 non-null    int8   
 4   institution         975 non-null    object 
 5   instructors         975 non-null    object 
 6   Level               975 non-null    int8   
 7   subject             975 non-null    object 
 8   language            975 non-null    object 
 9   subtitles           975 non-null    object 
 10  course_effort       975 non-null    object 
 11  course_length       975 non-null    int32  
 12  price               975 non-null    float64
 13  course_description  975 non-null    object 
 14  course_syllabus     975 non-null    object 
 15  course_url          975 non-null    object 
dtypes: float

In [45]:
edx_courses.describe()

,n_enrolled,course_type,Level,course_length,price
count,9.750000e+02,975.000000,975.000000,975.000000,975.000000
mean,4.670523e+04,0.940513,1.547692,6.705641,100.465497
std,1.128239e+05,0.236656,0.653165,3.150327,69.572175
min,0.000000e+00,0.000000,0.000000,1.000000,5.000000
25%,8.529500e+03,1.000000,1.000000,4.000000,49.000000
50%,1.968700e+04,1.000000,2.000000,6.000000,79.000000
75%,4.891850e+04,1.000000,2.000000,8.000000,149.000000
max,2.442271e+06,1.000000,2.000000,18.000000,450.000000


Now the whole dataset is cleaned and even.

In [30]:
# subjects with the most courses in edx(in ascending order)
edx_courses['subject'].value_counts()

Computer Science                166
Business & Management           164
Data Analysis & Statistics       72
Humanities                       64
Engineering                      58
Social Sciences                  51
Economics & Finance              46
Communication                    39
Biology & Life Sciences          35
Language                         34
Medicine                         31
Math                             31
Education & Teacher Training     29
History                          20
Physics                          18
Environmental Studies            16
Art & Culture                    12
Health & Safety                  11
Science                          10
Law                              10
Design                            8
Energy & Earth Sciences           8
Electronics                       7
Music                             7
Philosophy & Ethics               6
Food & Nutrition                  6
Architecture                      5
Chemistry                   

In [31]:
# export the cleaned dataset(edx_courses) to a csv file
edx_courses_clean = edx_courses.to_csv("edx_courses_clean.csv")